In [1]:
from __future__ import division
import _config
import sys, os, fnmatch, datetime, subprocess, imp
import numpy as np
from collections import defaultdict
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

inp_dir = _config.OUT_PLACE

### DESCRIPTION: UMI count in genomic dataset - initial plasmid integration should not have UMI duplications
### or miss-pairings between gRNA, UMI, target 

### b7 outputs does not check matches between gRNA and anything except for the read - need to make sure UMIs and SAs are also matched correctly
### preliminary: make sure that UMIs are unique to one ID# 
### 

In [2]:
import re

In [3]:
match_dct = {
    'A':'T',
    'C':'G',
    'T':'A',
    'G':'C',
}

In [4]:
b3_dir = os.path.join(inp_dir, 'b3_align_unspliced_reads_510/')
b6_dir = os.path.join(inp_dir, 'b6_spliced_data_processing_510/')
b7_dir = os.path.join(inp_dir, 'b7_genomic_data_processing_510/')

In [5]:
# just check the 0 directory for now
unspliced_p2l_1 = os.path.join(b3_dir, '190124Gif_D19-555/0/')
gdna_p2l_1 = os.path.join(b7_dir, '190510Gif_D19-212027/0/')
spliced_p2l_1 = os.path.join(b6_dir, '190124Gif_D19-558/0/')

In [6]:
def reverse_complement(s):
    # probably need reverse-reverse complemeent so we can start matching from the back rather than the front?
    # dunno this is purely a reverse complement 
    if len(s) == 0:
        return ''
    return match_dct[s[-1:]] + reverse_complement(s[:-1])

In [6]:
def umi_count(directory):
    #reads included?
    #how does drop duplicates check duplicated strings - substring or the entire string
    acc = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file != 'short.csv' and 'checkpoint' not in file:
                file_df = pd.read_csv(os.path.join(directory, file), sep = '\n')
                data = file_df.values 
                data = np.insert(data,0, '>1')
                data = np.reshape(data, (-1,3))
                data = pd.DataFrame(data, columns = ('extra','UMI', 'ID'))
                data = data.drop_duplicates(('extra','UMI', 'ID'))
                acc = acc + data.shape[0]
    return acc

In [13]:
umi_count(gdna_p2l_1)

238993

In [11]:
file_df = pd.read_csv(os.path.join(gdna_p2l_1_p2l_1, "AAAAAA.txt"))
file_df

,>1
0,AAAAAATACATATTT
1,5618
2,>1
3,AAAAAATGGCTCATA
4,8563
5,>1
6,AAAAAAAAAATCTTC
7,1049
8,>1
9,AAAAAATAGAGTAGA


In [10]:
def duplprocess_genomic(directory):
    data_df = pd.DataFrame(columns = ('UMI','ID'))
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file != 'short.csv' and 'checkpoint' not in file:
                file_df = pd.read_csv(os.path.join(directory, file), sep = '\n')
                data = file_df.values 
                data = np.insert(data,0, '>1')
                data = np.reshape(data, (-1,3))
                data = np.delete(data, np.s_[0:1], 1)
                data = pd.DataFrame(data, columns = ('UMI','ID'))
                duplicated = data[data.duplicated('UMI')]
                r = duplicated.groupby('UMI',as_index = False).apply(lambda x: x.drop_duplicates('ID'))
                for index, group in r.groupby('UMI',as_index=False):
                    if len(group) > 1:
                        data_df = data_df.append(group)
    return data_df

In [9]:
data = process_genomic(gdna_p2l_1)

#average redundacy for different lengths - similar number --> diversity (undersampling UMIs)

data.groupby(data.UMI.str.len()).apply(lambda x: x.ID.nunique())

UMI
15    2088
dtype: int64

In [8]:
duplprocess_genomic(gdna_p2l_1)

,UMI,ID
"(3, 16)",AAAATAGAAGACAAA,6040
"(3, 64)",AAAATAGAAGACAAA,4065
"(6, 22)",AAAATGTATAGAACT,284
"(6, 29)",AAAATGTATAGAACT,1389
"(2, 29)",AAAACAATAACGATG,2995
"(2, 69)",AAAACAATAACGATG,3045
"(10, 12)",AAAACTTCCAAATTT,5685
"(10, 72)",AAAACTTCCAAATTT,8926
"(10, 75)",AAAACTTCCAAATTT,3867
"(11, 23)",AAAACTTGGTAACCG,7674


In [ ]:
def process(directory):
    data_df = pd.DataFrame(columns = ('UMI','ID'))
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file != 'short.csv':
                file_df = pd.read_csv(os.path.join(directory, file), sep = '\n')
                data = file_df.values 
                data = np.insert(data,0, '<1')
                try:
                    data = np.reshape(data, (-1,8))
                    data = np.delete(data, np.s_[0:1], 1)
                    data = np.delete(data, np.s_[3:7], 1)
                    data = pd.DataFrame(data, columns = ('UMI','ID'))
                    data_df1 = data_df1.append(data)
                except:
                    print('why')
                    print(file)

In [57]:
data_df = pd.DataFrame(data = data, columns = ('UMI','ID','splice_type'))

In [58]:
data_df.UMI.value_counts()

AAAAAAGCATCATCTGGC    15
AAAAAAACTTGTTCAGGC     7
AAAAAAAGACTTAACGGC     4
AAAAAACAATCAGAGGCG     3
AAAAAAAACCCGACAGGC     3
AAAAAACAATATATAGGC     3
AAAAAATCCAGTGTTGGC     2
AAAAAAGGCTACTGGGGC     2
AAAAAACGTACGGCAGGC     2
AAAAAATTTTATATGGGC     2
AAAAAAGCATCATCTGG      2
AAAAAAACTCACGCGGG      2
AAAAAAACTCACGCGGGC     2
AAAAAATCTTTTGCTGGC     2
AAAAAACAATCAGAGGC      2
AAAAAAGAAAACCCGGGC     2
AAAAAAGAGTGATGGGGC     2
AAAAAACAGCTCTATGGC     2
AAAAAATTCTAATTTGGC     2
AAAAAATCATAAGTGGG      2
AAAAAATCATAAGTGGGC     2
AAAAAACATCACTGAGG      1
AAAAAACACAGGCAGGG      1
AAAAAATGGACTGAGGGC     1
AAAAAAGCGCCAATCGGC     1
AAAAAATGGGCGAATGGC     1
AAAAAAGTTGGCTAGGGC     1
AAAAAATTATACGGAGGC     1
AAAAAATCATATTGAGGC     1
AAAAAATGAACCCTCGGC     1
                      ..
AAAAAAGGACACATGGGC     1
AAAAAACGGCATGACGGC     1
AAAAAAACAAGGAATGGC     1
AAAAAAAGCAGACCCGGC     1
AAAAAAGGACACATGGG      1
AAAAAATTATCTTCTGGC     1
AAAAAAAGATTAGTGGGC     1
AAAAAAACTTGTTCAGG      1
AAAAAATCAAGGCATGGC     1


In [93]:
data_df1 = pd.DataFrame(columns = ('UMI','ID','splice_type'))
for root, dirs, files in os.walk(spliced_p2l_1):
    for file in files:
        if file != 'short.csv':
            file_df = pd.read_csv(os.path.join(spliced_p2l_1, file), sep = '\n')
            data = file_df.values 
            data = np.insert(data,0, '<1')
            try:
                data = np.reshape(data, (-1,8))
                data = np.delete(data, np.s_[0:1], 1)
                data = np.delete(data, np.s_[3:7], 1)
                data = pd.DataFrame(data, columns = ('UMI','ID','splice_type'))
                data_df1 = data_df1.append(data)
            except:
                print('why')
                print(file)

KeyboardInterrupt: 

In [90]:
# for folder 0, less than 15 percent have more than 2 instances of the same UMI 
data_df.UMI = data_df[0]
data_df.ID = data_df[1]
counts = data_df.UMI.value_counts()
ones = list(filter(lambda x: x < 3, counts))
len(ones)
print(str(len(ones)/398111))

0.8459349276960446


In [106]:
data_df_copy.UMI = data_df_copy.UMI.apply(lambda x: x[0:15])

In [110]:
counts = data_df_copy.UMI.value_counts()
ones = list(filter(lambda x: x < 2, counts))
len(ones)
print(str(len(ones)/398111))

0.5095940579386151


In [100]:
cas9_df0 = pd.DataFrame(columns = ('UMI','ID'))
for root, dirs, files in os.walk(unspliced_p2l_1):
    for file in files:
        if file != 'short.csv':
            file_df = pd.read_csv(os.path.join(unspliced_p2l_1, file), sep = '\n')
            data = file_df.values 
            data = np.insert(data,0, '<1')
            data = np.reshape(data, (-1,7))
            data = np.delete(data, np.s_[0:1], 1)
            data = np.delete(data, np.s_[2:7], 1)
            data = pd.DataFrame(data, columns = ('UMI','ID'))
            cas9_df0 = cas9_df0.append(data)

In [103]:
counts = cas9_df0.UMI.value_counts()
ones = list(filter(lambda x: x < 2, counts))
len(ones)
print(str(len(ones)/732882))

0.4212410729148758
